In [7]:
!pip install gen-dedup

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.9 MB/s eta 0:00:00
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 12.2 MB/s eta 0:00:0000:01
Using cached tqdm-4.66.5-py3-none-any.whl (78 kB)
   ━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/797.1 MB 32.6 MB/s eta 0:00:23^C
   ━━━╸━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['WANDB_MODE'] = 'disabled'

In [2]:
from datasets import load_dataset

# 1. load dataset
ds = load_dataset('cardiffnlp/tweet_eval', 'hate', split='train')
ds = ds.select_columns(['text'])
ds = ds.rename_column('text', 'sentence')

In [3]:
from keybert import KeyBERT

# 2. generate two keywords. Other keyword extraction models can also be used, such as tfidf, LLMs, etc.
keybert = KeyBERT()
ds = ds.map(lambda x: {'labels': " ".join([k[0] for k in keybert.extract_keywords(x['sentence'].lower())[:2]]), 'sentence': x['sentence'].lower()})


In [4]:
for obj in ds:
    print(obj)
    break


{'sentence': '@user nice new signage. are you not concerned by beatlemania -style hysterical crowds crongregating on you…', 'labels': 'signage crowds'}


In [5]:
from gen_dedup import GenDedup

gd = GenDedup('google/flan-t5-small')
gd.fit(ds, output_dir='./hate-dedup')

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

wandb: WARNING Disabling the wandb service is deprecated as of version 0.18.0 and will be removed in future versions. 
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,7.133300
20,5.306700
30,4.604000
40,3.911500
50,3.619200
60,3.565100
70,3.116100
80,2.804100
90,2.914800
100,2.558200


In [6]:
gd.deduplicate('hate-dedup', max_label_words=2)

  0%|          | 0/9000 [00:00<?, ?it/s]/data/lxm/miniconda3/envs/py310/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 9000/9000 [07:55<00:00, 18.91it/s]


100 duplicate text detected!
8900 nonduplicate text detected!
Duplicate text has been saved to hate-dedup/duplicate.jsonl
Nonduplicate text has been saved to hate-dedup/nonduplicate.jsonl
